DATA CREATION

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)
n = 500

orders = pd.DataFrame({
    'Order_ID': range(1, n+1),
    'Customer_ID': np.random.randint(100, 150, size=n),
    'Product_ID': np.random.randint(1000, 1020, size=n),
    'Quantity': np.random.randint(1, 5, size=n),
    'Order_Date': pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-06-30"), size=n))
})
orders.to_csv("orders.csv", index=False)


In [2]:
products = pd.DataFrame({
    'Product_ID': range(1000, 1020),
    'Product_Name': [f"Item_{i}" for i in range(20)],
    'Price': np.round(np.random.uniform(10, 100, 20), 2)
})
products.to_csv("products.csv", index=False)


In [3]:
customers = pd.DataFrame({
    'Customer_ID': range(100, 160),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], size=60),
    'Signup_Date': pd.to_datetime(np.random.choice(pd.date_range("2022-01-01", "2023-01-01"), size=60))
})
customers.to_csv("customers.csv", index=False)


## Tasks:
1. Clean:

    - Check for and handle duplicates or nulls.
    - Ensure data types are correct (Order_Date, Signup_Date)

2. Merge:

    - Join all 3 files into 1 master_df (on Customer_ID, Product_ID)
    - Create Revenue = Quantity × Price

3. Enrich:

    - Create new columns:
        - Days_Since_Signup = Order_Date − Signup_Date
        - Revenue_Category = low (<100), medium (100–250), high (>250)

4. Summarize:

    - Total revenue by Region
    - Most sold product by quantity
    - Average revenue per order



In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_customer = pd.read_csv("customers.csv")
df_orders = pd.read_csv("orders.csv")
df_products = pd.read_csv("products.csv")

In [4]:
df_orders.info(), df_products.info(), df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Order_ID     500 non-null    int64 
 1   Customer_ID  500 non-null    int64 
 2   Product_ID   500 non-null    int64 
 3   Quantity     500 non-null    int64 
 4   Order_Date   500 non-null    object
dtypes: int64(4), object(1)
memory usage: 19.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product_ID    20 non-null     int64  
 1   Product_Name  20 non-null     object 
 2   Price         20 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 608.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       -------------

(None, None, None)

In [5]:
df_orders.isnull().sum(), df_products.isnull().sum(), df_customer.isnull().sum()

(Order_ID       0
 Customer_ID    0
 Product_ID     0
 Quantity       0
 Order_Date     0
 dtype: int64,
 Product_ID      0
 Product_Name    0
 Price           0
 dtype: int64,
 Customer_ID    0
 Region         0
 Signup_Date    0
 dtype: int64)

In [8]:
df_orders.duplicated().sum(), df_products.duplicated().sum(), df_customer.duplicated().sum()

(np.int64(0), np.int64(0), np.int64(0))

In [ ]:
df_orders['Order_Date'] = pd.to_datetime(df_orders['Order_Date'])
df_customer['Signup_Date'] = pd.to_datetime(df_customer['Signup_Date'])

In [21]:
df_temp = pd.merge(df_orders, df_products, on= 'Product_ID')
df_master = pd.merge(df_temp, df_customer, on='Customer_ID')

In [23]:
df_master['Revenue'] = df_master['Quantity'] * df_master['Price']

In [24]:
df_master

,Order_ID,Customer_ID,Product_ID,Quantity,Order_Date,Product_Name,Price,Region,Signup_Date,Revenue
0,1,138,1015,1,2023-01-12,Item_15,66.84,West,2022-08-04,66.84
1,2,128,1006,1,2023-01-19,Item_6,53.80,North,2022-09-02,53.80
2,3,114,1003,1,2023-04-11,Item_3,62.08,West,2022-06-05,62.08
3,4,142,1000,1,2023-02-13,Item_0,98.89,East,2022-08-05,98.89
4,5,107,1004,3,2023-02-18,Item_4,49.48,North,2022-06-24,148.44
...,...,...,...,...,...,...,...,...,...,...
495,496,104,1013,4,2023-05-28,Item_13,28.96,South,2022-10-11,115.84
496,497,111,1018,3,2023-02-06,Item_18,74.42,North,2022-04-30,223.26
497,498,115,1005,2,2023-01-04,Item_5,75.27,North,2022-07-26,150.54
498,499,125,1000,2,2023-04-18,Item_0,98.89,North,2022-04-29,197.78


In [ ]:
df_master['Days_Since_Signup'] = (df_master['Order_Date'] - df_master['Signup_Date']).dt.days


In [28]:
bins = [float('-inf'), 100, 250, float('inf')]
labels = ['Low', 'medium', 'High']

df_master['Revenue_Category'] = pd.cut(df_master['Revenue'], bins=bins, labels=labels, right=False)

In [30]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Order_ID           500 non-null    int64         
 1   Customer_ID        500 non-null    int64         
 2   Product_ID         500 non-null    int64         
 3   Quantity           500 non-null    int64         
 4   Order_Date         500 non-null    datetime64[ns]
 5   Product_Name       500 non-null    object        
 6   Price              500 non-null    float64       
 7   Region             500 non-null    object        
 8   Signup_Date        500 non-null    datetime64[ns]
 9   Revenue            500 non-null    float64       
 10  Days_Since_Signup  500 non-null    int64         
 11  Revenue_Category   500 non-null    category      
dtypes: category(1), datetime64[ns](2), float64(2), int64(5), object(2)
memory usage: 43.7+ KB


In [32]:
df_master.groupby('Region')['Revenue'].sum()

Region
East     22094.92
North    20522.74
South    18989.25
West     16061.42
Name: Revenue, dtype: float64

In [39]:
df_master.groupby('Product_Name')['Quantity'].sum().sort_values(ascending=False).head(7)


Product_Name
Item_4     100
Item_16     85
Item_18     69
Item_14     68
Item_2      68
Item_1      66
Item_3      65
Name: Quantity, dtype: int64

In [47]:
df_master['Order_ID'].is_unique

True

In [48]:
# mean revenue per order and sum of revenue per order is equal as 'Order_ID' is unique
df_master.groupby('Order_ID')['Revenue'].mean() 

Order_ID
1       66.84
2       53.80
3       62.08
4       98.89
5      148.44
        ...  
496    115.84
497    223.26
498    150.54
499    197.78
500    132.14
Name: Revenue, Length: 500, dtype: float64

In [50]:
df_master.to_csv("df_master.csv", index=False)